Colab notebook for finding solution by brute-force for small VEHICLE_NUM and VERTICES_NUM

In [ ]:
import random
import time
import math
import pickle
import itertools
import threading
from typing import List

from numpy import Infinity

from settings import VEHICLE_NUM,VERTICES_NUM

Point = tuple[int, int]

vertices: List[Point] = [(0, 0)]
distance_matrix: List[List[float]] = []

In [ ]:
with open('vertices', 'rb') as vt:
    vertices = pickle.load(vt)
with open('distance', 'rb') as dt:
    distance_matrix = pickle.load(dt)

In [ ]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

In [ ]:
pool = [i+1 for i in range(VERTICES_NUM)]
for i in range(VEHICLE_NUM-1):
    pool.append(-1)
permutations = set(itertools.permutations(pool))
permutations = list(permutations)
p_list = split(permutations,10)

In [ ]:
result =[]
def find_min(permutations):
  min_s = Infinity
  s = ()
  for i,p in enumerate(permutations):
      # print('\r',f'{i} - {len(permutations)}',end='')
      max: float = 0
      prev = 0
      sum = 0
      for current in p:
          if current == -1:
              if sum > max:
                  max = sum
              sum = 0
              prev = 0
              continue
          sum += distance_matrix[current][prev]
          prev = current
      if sum > max:
          max = sum
      if min_s > max:
        min_s = max
        s = p
  result.append((s,min_s))

In [ ]:
threadlist = []
for p in p_list:
  t = threading.Thread(target=find_min, args=(p,))
  t.start()
  threadlist.append(t)
for i,t in enumerate(threadlist):
  t.join()
  print(f'thread {i} finished')

In [ ]:
final = min(r[1] for r in result)

In [ ]:
with open('output', 'w') as o:
    for r in result:
      if r[1] == final:
        o.write(str(r)+'\n')
        break
        